# 1. Creating Datasets

In [1]:
import torch

config = {
    "data_dir": "./../../ML_project/FreiHAND",
    "epochs": 1000,
    "batch_size": 48,
    "batches_per_epoch": 50,
    "batches_per_epoch_val": 20,
    "learning_rate": 0.1,
    "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
}

In [2]:
from settings import TRAIN_NAME, VALIDATION_NAME
from model.dataset import FreiHANDDataset, FreiHANDDataLoader

Training set

In [3]:
train_dataset = FreiHANDDataset(set_type=TRAIN_NAME)
train_dataset

FreiHAND [Training - 26000 items]

In [4]:
train_dataloader = FreiHANDDataLoader(
    dataset=train_dataset,
    batch_size=config["batch_size"],
    shuffle=True,
    drop_last=True,
    num_workers=2
)
train_dataloader

FreiHANDDataLoader [Training - Batch size: 48 - Length: 541]

Validation set

In [5]:
val_dataset = FreiHANDDataset(set_type=VALIDATION_NAME)
val_dataset

FreiHAND [Validation - 5000 items]

In [6]:
val_dataloader = FreiHANDDataLoader(
    dataset=val_dataset,
    batch_size=config["batch_size"],
    shuffle=True,
    drop_last=True,
    num_workers=2
)
val_dataloader

FreiHANDDataLoader [Validation - Batch size: 48 - Length: 104]

# 2. Models

Creating the ANN:

In [7]:
from model.network import ShallowUNet


In [8]:
model = ShallowUNet(in_channel=3, out_channel=21)
model = model.to(config["device"])

Creating the loss criterion

In [9]:
from model.network import IoULoss

In [10]:
criterion = IoULoss()

Creating the optimizer

In [11]:
import torch.optim as optim

In [12]:
optimizer = optim.SGD(model.parameters(), lr=config["learning_rate"])
optimizer

SGD (
Parameter Group 0
    dampening: 0
    foreach: None
    lr: 0.1
    maximize: False
    momentum: 0
    nesterov: False
    weight_decay: 0
)

Creating the scheduler

In [13]:
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer=optimizer, factor=0.5, patience=20, verbose=True, threshold=0.00001
)
scheduler

# 3. Training

In [14]:
from model.trainer import Trainer

trainer = Trainer(model, criterion, optimizer, config, scheduler)
trainer

In [ ]:
model = trainer.train(train_dataloader, val_dataloader)

INFO: Progress: [0/541] - 0.00%
INFO: Progress: [1/541] - 0.18%
